## Preprocessing

In [24]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
import os

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [25]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns= ['EIN', 'NAME'], axis=1, inplace=True)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [26]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [27]:
# Look at APPLICATION_TYPE value counts for binning
app_type_count = application_df['APPLICATION_TYPE'].value_counts()
app_type_count

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [28]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_count[app_type_count < 500].index)


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [29]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df['CLASSIFICATION'].value_counts()
classification_count.head(15)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
Name: count, dtype: int64

In [30]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_greater1 = classification_count.loc[classification_count > 1]
classification_greater1.head(10)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
Name: count, dtype: int64

In [31]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_count[classification_count < 350].index)


# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64

In [32]:
# Convert categorical data to numeric with `pd.get_dummies`
numeric_data = pd.get_dummies(application_df)
numeric_data = numeric_data.astype(int)
numeric_data.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [33]:
# Split our preprocessed data into our features and target arrays
y = numeric_data ['IS_SUCCESSFUL']
X = numeric_data.drop(columns='IS_SUCCESSFUL', axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=78)

In [34]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

First attempt. 

A loss value of 55 indicates that the model can be further optimized.

The accuracy percent shows that 72% of the model's predicted values align with the true values in the original dataset.

In [35]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features_total = len(X_train_scaled[0])
hidden_nodes_layer_1 = 80
hidden_nodes_layer_2 = 30

# Initialize the Sequential model
nn = tf.keras.models.Sequential()

# Explicit Input layer
nn.add(tf.keras.layers.Input(shape=(input_features_total,)))

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer_1, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer_2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                 │ (None, 80)             │         3,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 30)             │         2,430 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,061 (23.68 KB)

 Trainable params: 6,061 (23.68 KB)

 Non-trainable params: 0 (0.00 B)

In [36]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [37]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 377us/step - accuracy: 0.7068 - loss: 0.5881
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 337us/step - accuracy: 0.7308 - loss: 0.5524
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 333us/step - accuracy: 0.7387 - loss: 0.5460
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 333us/step - accuracy: 0.7267 - loss: 0.5554
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 331us/step - accuracy: 0.7319 - loss: 0.5513
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 334us/step - accuracy: 0.7327 - loss: 0.5484
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 331us/step - accuracy: 0.7308 - loss: 0.5505
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 330us/step - accuracy: 0.7365 - loss: 0.5442
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 330us/step - accuracy: 0.7338 - loss: 0.5467
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 329us/step - accuracy: 0.7327 - loss: 0.5493
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 342us/step - accuracy: 0.7387 - loss: 0.5404
Epoch 12/100
804/80

In [38]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 444us/step - accuracy: 0.7248 - loss: 0.5558
Loss: 0.5558362603187561, Accuracy: 0.724781334400177


In [42]:
#nn.save('Models/AlphabetSoupCharity3.h5')
#Warning: this file format is considered legacy. We recommend using instead the native Keras format,

In [43]:
# Save the model using the native Keras format

nn.save('Models/AlphabetSoupCharity1.keras')


Second attempt.

Adding a third hidden layer.

In [44]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = X_train_scaled.shape[1]
hidden_nodes1=7
hidden_nodes2=14
hidden_nodes3=21

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes1, input_dim=input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes2, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 7)              │           315 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 14)             │           112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 21)             │           315 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │            22 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 764 (2.98 KB)

 Trainable params: 764 (2.98 KB)

 Non-trainable params: 0 (0.00 B)

In [46]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [47]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,validation_split=0.15, epochs=200)

Epoch 1/200
684/684 ━━━━━━━━━━━━━━━━━━━━ 1s 506us/step - accuracy: 0.6732 - loss: 0.6308 - val_accuracy: 0.7056 - val_loss: 0.5710
Epoch 2/200
684/684 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step - accuracy: 0.7250 - loss: 0.5613 - val_accuracy: 0.7175 - val_loss: 0.5656
Epoch 3/200
684/684 ━━━━━━━━━━━━━━━━━━━━ 0s 415us/step - accuracy: 0.7313 - loss: 0.5553 - val_accuracy: 0.7199 - val_loss: 0.5632
Epoch 4/200
684/684 ━━━━━━━━━━━━━━━━━━━━ 0s 414us/step - accuracy: 0.7301 - loss: 0.5580 - val_accuracy: 0.7181 - val_loss: 0.5627
Epoch 5/200
684/684 ━━━━━━━━━━━━━━━━━━━━ 0s 442us/step - accuracy: 0.7315 - loss: 0.5547 - val_accuracy: 0.7191 - val_loss: 0.5613
Epoch 6/200
684/684 ━━━━━━━━━━━━━━━━━━━━ 0s 416us/step - accuracy: 0.7303 - loss: 0.5553 - val_accuracy: 0.7232 - val_loss: 0.5596
Epoch 7/200
684/684 ━━━━━━━━━━━━━━━━━━━━ 0s 419us/step - accuracy: 0.7271 - loss: 0.5516 - val_accuracy: 0.7235 - val_loss: 0.5607
Epoch 8/200
684/684 ━━━━━━━━━━━━━━━━━━━━ 0s 432us/step - accuracy: 0.7327 - loss: 0

In [48]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 288us/step - accuracy: 0.7256 - loss: 0.5568
Loss: 0.5567882657051086, Accuracy: 0.7255976796150208


Third attempt

Adding a third layer and use increase use of activation 'sigmoid'

In [49]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 8
hidden_nodes_layer3= 6

nn_model2 = tf.keras.models.Sequential()

# First hidden layer
nn_model2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1,
             input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_model2.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn_model2.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn_model2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn_model2.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_13 (Dense)                │ (None, 10)             │           450 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 8)              │            88 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 6)              │            54 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 599 (2.34 KB)

 Trainable params: 599 (2.34 KB)

 Non-trainable params: 0 (0.00 B)

In [50]:
# Compile the model
nn_model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [51]:
# Train the model
fit_model2 = nn_model2.fit(X_train_scaled, y_train, epochs=30)

Epoch 1/30
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 365us/step - accuracy: 0.5790 - loss: 0.6903
Epoch 2/30
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 354us/step - accuracy: 0.7317 - loss: 0.5707
Epoch 3/30
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 363us/step - accuracy: 0.7305 - loss: 0.5630
Epoch 4/30
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 358us/step - accuracy: 0.7278 - loss: 0.5600
Epoch 5/30
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 355us/step - accuracy: 0.7304 - loss: 0.5575
Epoch 6/30
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 372us/step - accuracy: 0.7328 - loss: 0.5585
Epoch 7/30
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 414us/step - accuracy: 0.7351 - loss: 0.5512
Epoch 8/30
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 371us/step - accuracy: 0.7382 - loss: 0.5492
Epoch 9/30
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 365us/step - accuracy: 0.7309 - loss: 0.5524
Epoch 10/30
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 393us/step - accuracy: 0.7264 - loss: 0.5578
Epoch 11/30
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 365us/step - accuracy: 0.7293 - loss: 0.5521
Epoch 12/30
804/804 ━━━━━━━━━━

In [52]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 430us/step - accuracy: 0.7279 - loss: 0.5542
Loss: 0.5541703104972839, Accuracy: 0.7279300093650818
